<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/19.%20Chapter19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Bab 19: Training and Deploying TensorFlow Models at Scale

Setelah Anda memiliki model yang bagus, langkah selanjutnya adalah membawanya ke lingkungan produksi. Bab ini membahas cara men-deploy model TensorFlow agar dapat digunakan oleh aplikasi lain dan cara melatih model secara efisien pada dataset besar menggunakan beberapa perangkat.

* **Men-deploy Model dengan TensorFlow Serving**:
    * **SavedModel Format**: Langkah pertama adalah mengekspor model Anda ke format **SavedModel**. Ini adalah format universal TensorFlow yang menyimpan arsitektur model (grafik komputasi), bobot, dan aset lainnya dalam sebuah direktori.
    * **TensorFlow Serving (TF Serving)**: Server produksi berperforma tinggi yang ditulis dalam C++. Fitur utamanya adalah:
        * Dapat melayani beberapa model atau versi model secara bersamaan.
        * Secara otomatis memuat versi model terbaru dari sebuah direktori.
        * Mendukung *batching* permintaan untuk meningkatkan *throughput* pada GPU.
        * Dapat di-deploy dengan mudah menggunakan **Docker**.
    * **Querying the Service**: Anda dapat berinteraksi dengan TF Serving melalui:
        * **REST API**: Mudah digunakan, berbasis JSON, dan kompatibel dengan hampir semua klien.
        * **gRPC API**: Lebih efisien karena menggunakan format biner Protobuf, cocok untuk data besar dan latensi rendah.

* **Deploy ke Platform Cloud**: Menggunakan layanan cloud seperti **Google Cloud AI Platform** menyederhanakan proses deployment, penskalaan, dan pemantauan. Anda cukup mengunggah SavedModel Anda ke Google Cloud Storage (GCS), lalu membuat model dan versi di AI Platform.

* **Deploy ke Perangkat Seluler/Embedded (TFLite)**:
    * **TensorFlow Lite (TFLite)** adalah kerangka kerja untuk menjalankan model pada perangkat dengan sumber daya terbatas.
    * Tujuannya adalah untuk mengurangi ukuran model dan latensi.
    * Teknik yang digunakan termasuk **kuantisasi** (*quantization*), di mana bobot dan aktivasi dikonversi dari float 32-bit menjadi integer 8-bit, yang secara dramatis mengurangi ukuran dan mempercepat inferensi.

* **Melatih Model dalam Skala Besar**:
    * **Menggunakan GPU**: Merupakan cara paling umum untuk mempercepat training. TensorFlow secara otomatis akan menempatkan komputasi pada GPU jika tersedia.
    * **Distribution Strategies API (`tf.distribute`)**: API sederhana dari TensorFlow untuk mendistribusikan training di beberapa perangkat atau server.
        * ***Data Parallelism***: Strategi yang paling umum, di mana model direplikasi di setiap perangkat, dan setiap replika dilatih pada *mini-batch* data yang berbeda.
        * ***MirroredStrategy***: Untuk training sinkron di beberapa GPU pada satu mesin.
        * ***MultiWorkerMirroredStrategy***: Untuk training sinkron di beberapa mesin (*multi-worker*).
        * ***ParameterServerStrategy***: Untuk training asinkron di mana parameter disimpan di server terpusat (*parameter servers*).

### 1. Mengekspor Model ke Format SavedModel
Setiap model Keras dapat dengan mudah disimpan dalam format ini.

```python
import tensorflow as tf
from tensorflow import keras
import os

# Misalkan kita punya model yang sudah dilatih
# (Gunakan model dari bab sebelumnya atau buat model dummy)
model = keras.models.Sequential([
    keras.layers.Dense(10, activation="relu", input_shape=[8]),
    keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer="sgd")
# model.fit(...)

# Tentukan path untuk menyimpan model
model_name = "my_keras_model"
model_version = "0001"
model_path = os.path.join(model_name, model_version)

# Simpan model
model.save(model_path)
```
Setelah menjalankan kode ini, direktori `my_keras_model/0001/` akan dibuat dengan semua file yang diperlukan.

### 2. Menjalankan TensorFlow Serving dengan Docker
Perintah ini harus dijalankan di terminal lokal Anda (bukan di Colab) setelah menginstal Docker.

```bash
# Ganti $ML_PATH dengan path ke direktori proyek Anda
# Perintah ini akan menjalankan TF Serving di Docker dan memuat model Anda
# docker run -it --rm -p 8501:8501 \
#    -v "$ML_PATH/my_keras_model:/models/my_keras_model" \
#    -e MODEL_NAME=my_keras_model \
#    tensorflow/serving
```

### 3. Meng-query TF Serving Melalui REST API
Setelah server berjalan, Anda dapat mengirim permintaan dari mana saja, termasuk dari notebook Colab.

```python
import json
import requests
import numpy as np

# Siapkan data input dummy
X_new = np.random.rand(3, 8).astype(np.float32)

# URL server (jika berjalan di mesin lokal)
SERVER_URL = 'http://localhost:8501/v1/models/my_keras_model:predict'

# Membuat permintaan POST
# Kita perlu mengubah data NumPy menjadi list Python agar bisa di-encode ke JSON
input_data_json = json.dumps({
    "instances": X_new.tolist()
})

try:
    response = requests.post(SERVER_URL, data=input_data_json)
    response.raise_for_status() # Lemparkan error jika status bukan 200
    response_data = response.json()
    y_pred = np.array(response_data["predictions"])
    print("Prediksi dari REST API:\n", y_pred)
except requests.exceptions.ConnectionError as e:
    print("Koneksi ke TF Serving gagal. Pastikan server Docker berjalan.")
    print(e)
```

### 4. Melatih di Beberapa GPU dengan `MirroredStrategy`
Jika notebook Colab Anda berjalan pada *runtime* GPU, Anda bisa mensimulasikan penggunaan beberapa GPU atau menjalankannya jika tersedia lebih dari satu.

```python
# Membuat strategi distribusi
# TensorFlow akan secara otomatis mendeteksi semua GPU yang tersedia
try:
    distribution = tf.distribute.MirroredStrategy()
    print("Jumlah perangkat:", distribution.num_replicas_in_sync)
except RuntimeError as e:
    print("MirroredStrategy hanya dapat digunakan dengan GPU atau TPU.")
    print(e)
    distribution = None

if distribution:
    # Semua pembuatan model dan kompilasi harus berada di dalam scope strategi
    with distribution.scope():
        mirrored_model = keras.models.Sequential([
            keras.layers.Dense(10, activation="relu", input_shape=[8]),
            keras.layers.Dense(1)
        ])
        mirrored_model.compile(loss="mse", optimizer="sgd")

    # Data dummy untuk training
    X_train_dist = np.random.rand(100, 8)
    y_train_dist = np.random.rand(100, 1)

    # Training akan didistribusikan secara otomatis
    # batch_size harus dapat dibagi rata dengan jumlah replika/GPU
    # history = mirrored_model.fit(X_train_dist, y_train_dist, epochs=10,
    #                                batch_size=32 * distribution.num_replicas_in_sync)
    print("\nModel dengan MirroredStrategy telah dibuat dan dikompilasi.")
    print("Proses training akan didistribusikan secara otomatis saat model.fit() dipanggil.")
```
`MirroredStrategy` membuat proses training terdistribusi menjadi sangat sederhana. Model dan semua variabelnya direplikasi di setiap GPU, dan gradien disinkronkan di setiap langkah training.
